# PANDAS: Uso eficiente de Pandas

## Intro

### Dataset

[Link al dataset](https://www.kaggle.com/datasets/rajugc/imdb-top-250-movies-dataset)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
top_250_movies = pd.read_csv('data/top_250_movies.csv', index_col='rank')

In [ ]:
top_250_movies.head(3)

### Querying 

In [ ]:
top_250_movies.certificate.value_counts()

In [ ]:
top_250_movies.loc[top_250_movies.year.eq(1995)].sort_values(by='rating', ascending=False)

### Gráficos

In [ ]:
top_250_movies.certificate.value_counts().plot(kind='bar', rot=45)
None

In [ ]:
import seaborn as sns

sns.histplot(data=top_250_movies, x='year', bins=np.arange(1920, 2025, 5))

## Pandas y Numpy

In [ ]:
top_250_movies.year

In [ ]:
top_250_movies.dtypes

## Optimización de datos

In [ ]:
top_250_movies.head(1)

### Chaining vs =

In [ ]:
top_250_movies = top_250_movies.drop(columns=['tagline', 'casts', 'directors', 'writers'])

top_250_movies.loc[:, 'year'] = top_250_movies.year * 2

top_250_movies.loc[:, 'first_letter'] = top_250_movies.name.str[0]

In [ ]:
top_250_movies

In [ ]:
top_250_movies = pd.read_csv('data/top_250_movies.csv', index_col='rank')

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    year=top_250_movies.year * 2,
    first_letter=top_250_movies.name.str[0],
)
)

### Memory usage

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.memory_usage(deep=True)
)

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.memory_usage(deep=True)
.sum()
)

In [ ]:
def mem_usage(df):
    if type(df) == pd.core.series.Series:
        kb = df.memory_usage(deep=True) / (2**10)
    else:
        kb = df.memory_usage(deep=True).sum() / (2**10)

    return round(kb)

In [ ]:
mem_usage(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
)

In [ ]:
top_250_movies.drop(
    columns=['tagline', 'casts', 'directors', 'writers']
).dtypes


In [12]:
np.iinfo(np.int64)

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

In [13]:
np.iinfo(np.int16)

iinfo(min=-32768, max=32767, dtype=int16)

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.astype({
    'year': 'int16'
})
.dtypes
)

In [ ]:
top_250_movies.rating

In [ ]:
np.iinfo(np.uint8)

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    rating=top_250_movies.rating * 10
)
.astype({
    'year': np.int16,
    'rating': np.uint8
})
.dtypes
)

In [ ]:
top_250_movies.certificate.head()

In [14]:
top_250_movies.certificate.unique()

array(['R', 'PG-13', 'Approved', 'PG', '18+', 'Not Rated', 'G', 'Passed',
       'Not Available', 'TV-PG', 'Unrated', 'X', '13+', 'TV-MA', 'GP'],
      dtype=object)

In [ ]:
mem_usage(top_250_movies.certificate)

In [ ]:
mem_usage(top_250_movies.certificate.astype('category'))

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    rating=top_250_movies.rating * 10,
)
.astype({
    'year': np.int16,
    'rating': np.uint8,
    'certificate': 'category',
})
)

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    rating=top_250_movies.rating * 10,
)
.astype({
    'year': np.int16,
    'rating': np.uint8,
    'certificate': 'category',
})
.dtypes
)

In [ ]:
top_250_movies.loc[:10, ['budget', 'box_office']]

In [ ]:
top_250_movies.budget.astype(np.int64)

In [ ]:
top_250_movies.budget.str.extract('(\d+)')

In [ ]:
top_250_movies.budget.str.extract('(\d+)').fillna(0)[0]

In [ ]:
def amount_to_millions(df, column):
    return (
        df
        [column]
        .str.extract('(\d+)')
        .fillna(0)
        .astype(float)
        / 1_000_000
    )[0]

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    rating=top_250_movies.rating * 10,
    budget=amount_to_millions(top_250_movies, 'budget'),
    box_office=amount_to_millions(top_250_movies, 'box_office'),
)
.astype({
    'year': np.int16,
    'rating': np.uint8,
    'certificate': 'category',
    'budget': np.float32,
    'box_office': np.float32,
})
)

In [ ]:
top_250_movies.genre

In [ ]:
top_250_movies.genre.value_counts()

In [ ]:
top_250_movies.genre[3]

In [ ]:
movie_genres = top_250_movies.genre.str.split(',')
genres = (set([genre for genre_list in movie_genres for genre in genre_list]))

In [ ]:
genres

In [ ]:
len(genres)

In [ ]:
top_250_movies.genre.memory_usage(deep=True)

In [ ]:
pd.Series(top_250_movies.index > 0).memory_usage(deep=True) * len(genres)

In [ ]:
top_250_movies.genre.str.contains('Action')

In [ ]:
#NO
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    rating=top_250_movies.rating * 10,
    budget=amount_to_millions(top_250_movies, 'budget'),
    box_office=amount_to_millions(top_250_movies, 'box_office'),
    Action=top_250_movies.genre.str.contains('Action'),
    Drama=top_250_movies.genre.str.contains('Drama'),
    .
    .
    .
)
.astype({
    'year': np.int16,
    'rating': np.uint8,
    'certificate': 'category',
    'budget': np.float32,
    'box_office': np.float32,
})
.dtypes
)

In [ ]:
def has_genre(df, genre):
    return df.genre.str.contains(genre)

(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    rating=top_250_movies.rating * 10,
    budget=amount_to_millions(top_250_movies, 'budget'),
    box_office=amount_to_millions(top_250_movies, 'box_office'),
    **{genre:has_genre(top_250_movies, genre) for genre in genres},
)
.drop(columns='genre')
.astype({
    'year': np.int16,
    'rating': np.uint8,
    'certificate': 'category',
    'budget': np.float32,
    'box_office': np.float32,
})
)

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    rating=top_250_movies.rating * 10,
    budget=amount_to_millions(top_250_movies, 'budget'),
    box_office=amount_to_millions(top_250_movies, 'box_office'),
    **{genre:has_genre(top_250_movies, genre) for genre in genres},
)
.drop(columns='genre')
.astype({
    'year': np.int16,
    'rating': np.uint8,
    'certificate': 'category',
    'budget': np.float32,
    'box_office': np.float32,
})
).loc[:, list(genres)].sum()

In [ ]:
top_250_movies.run_time

In [ ]:
def run_time_to_minutes(df):
    return (
        df.run_time
        .str.findall('[0-9]+')
        .apply(pd.Series, dtype='object')
        .fillna(0)
        .astype('int16')
        .assign(run_time=lambda df: (df[0] * 60) + df[1])
    ).run_time

In [ ]:
(
top_250_movies
.drop(columns=['tagline', 'casts', 'directors', 'writers'])
.assign(
    rating=top_250_movies.rating * 10,
    budget=amount_to_millions(top_250_movies, 'budget'),
    box_office=amount_to_millions(top_250_movies, 'box_office'),
    run_time=run_time_to_minutes(top_250_movies),
    **{genre:has_genre(top_250_movies, genre) for genre in genres},
)
.drop(columns='genre')
.astype({
    'year': np.int16,
    'rating': np.uint8,
    'certificate': 'category',
    'budget': np.float32,
    'box_office': np.float32,
    'run_time': np.int16
})
)

In [ ]:
def clean_data(input_df):
    def run_time_to_minutes(df):
        return (
            df.run_time
            .str.findall('[0-9]+')
            .apply(pd.Series, dtype='object')
            .fillna(0)
            .astype('int16')
            .assign(run_time=lambda df: (df[0] * 60) + df[1])
        ).run_time
    
    def has_genre(df, genre):
        return df.genre.str.contains(genre)
    
    def amount_to_millions(df, column):
        return (
            df
            [column]
            .str.extract('(\d+)')
            .fillna(0)
            .astype(float)
            / 1_000_000
        )[0]
    
    movie_genres = input_df.genre.str.split(',')
    genres = (set([genre for genre_list in movie_genres for genre in genre_list]))

    return (
        input_df
        .drop(columns=['tagline', 'casts', 'directors', 'writers'])
        .assign(
            rating=input_df.rating * 10,
            budget=amount_to_millions(input_df, 'budget'),
            box_office=amount_to_millions(input_df, 'box_office'),
            run_time=run_time_to_minutes(input_df),
            **{genre:has_genre(input_df, genre) for genre in genres},
        )
        .drop(columns='genre')
        .astype({
            'year': np.int16,
            'rating': np.uint8,
            'certificate': 'category',
            'budget': np.float32,
            'box_office': np.float32,
            'run_time': np.int16
        })
    )

In [ ]:
movies_refined = clean_data(top_250_movies)

In [ ]:
movies_refined

In [ ]:
mem_usage(movies_refined)

### Key Takeaways

- Vale la pena invertir un poco de tiempo en optimizar los tipos de nuestros dataframes

- Ojo con re-asignar nuestros dataframes, el chaining esta OP

- Los datasets crudos suelen ser feos